#### This notebook is to *numerically* calculate multitude of random distribution (m.r.d.) expected for $(\phi,\psi)$ diffraction orientations

- Dependents

In [ ]:
pwd

In [ ]:
ls

In [ ]:
%pylab inline

import TX, os, shutil
from TX import cmb, euler
euler = euler.euler
path_home=os.getcwd()
# cmb.random

path_load='/Users/yj/repo/rs_pack/src/src_fortran/'
os.chdir(path_load)
import cs
reload(cs)
os.chdir(path_home)

In [ ]:
### Define random orientation
grains=cmb.random(ngrain=1000,phi1=360, phi2=360, phi=360)
print grains.shape

In [ ]:
## rotation matrices for indivial discrete orientations
a_mat=[]
for i in xrange(len(grains)):
    gr=grains[i]
    ph,th,tm = gr[:3]
    a=euler(ph=grains[0][0], th=grains[0][1], tm=grains[0][2],echo=False)
    a_mat.append(a)

In [ ]:
hkls,n=cs.cubic_eqvect(v=[2,1,1])
hkls=hkls.T
hkls=hkls[:n,:]